In [1]:
import numpy as np
import pandas as pd
import wordfreq 
global possibles, must_haves, all_wordle_words

#Only Acceptable Wordle Guesses
url = "https://raw.githubusercontent.com/lightbulbsocks/Rank-One-Wordle/main/All%20Acceptable%20Wordle%20Words.csv"

df1 = pd.read_csv(url,engine='python')

all_wordle_words = []
for i in range(0,np.size(df1)):
        all_wordle_words.append(df1.iat[i,0].upper())

In [8]:
possible_words = all_wordle_words.copy()

freqs = []

for word in possible_words:
     freq = wordfreq.word_frequency(word, 'en')
     freqs.append(freq)

def scale_range (input, min, max):
    input += -(np.min(input))
    input /= np.max(input) / (max - min)
    input += min
    return input

def sig(x):
     return 1/(1 + np.exp(-x))

freqs = np.array(freqs)     

freqs = scale_range(freqs,-10, 10) 

F = [sig(x) for x in freqs]

F

[4.5752647771218716e-05,
 4.541100102226934e-05,
 4.5571463424499514e-05,
 4.8228031404357196e-05,
 4.6381396365734426e-05,
 4.5556215663158314e-05,
 4.682698998697142e-05,
 4.546845884299202e-05,
 4.5888825772084255e-05,
 4.607567706794352e-05,
 0.9999546021312976,
 0.0001598626207331622,
 6.533897005248447e-05,
 4.631492400926818e-05,
 4.583401384648961e-05,
 4.548259003650569e-05,
 6.482044314419496e-05,
 4.916306688776279e-05,
 4.57213075319898e-05,
 4.898710926056517e-05,
 4.617858539965291e-05,
 4.8509364808535884e-05,
 6.145086730025263e-05,
 4.6640808101782675e-05,
 5.136541917366422e-05,
 4.724673867633643e-05,
 4.576832594550325e-05,
 7.505523765596979e-05,
 4.555258599577965e-05,
 4.550216355485651e-05,
 4.567943261557092e-05,
 4.555258599577965e-05,
 0.5692727137826772,
 0.002453047405076652,
 4.555984561973867e-05,
 4.563214193161609e-05,
 7.5957626086998e-05,
 4.633707087004563e-05,
 4.955634984297749e-05,
 4.545650510840292e-05,
 4.6860581565047715e-05,
 4.55881692090508

In [9]:
dic = dict()

i = 0

for word in possible_words:
    dic[word] = F[i]
    i += 1

In [12]:
possible_words = all_wordle_words.copy()

def word_to_vector_loc_mat(word):
     array = np.zeros((130,1))
     for i in range(0,5):
          array[ord(word[i]) - 65 + i*26] = 1
    #  freq = wordfreq.word_frequency(word, 'en')
    #  return array*freq
     return array*dic[word]
   

def rank_one_approx(possible_words):
    A = word_to_vector_loc_mat(possible_words[0])
    for word in possible_words:
        A = np.concatenate((A,word_to_vector_loc_mat(word)),1)                  
    
    A = np.delete(A,0,1)    
    AAt = np.matmul(A,np.matrix.transpose(A))
    w,v = np.linalg.eigh(AAt)
    dom1 = np.abs(v[:,-1])
    return dom1, A

def find_best_guess(possible_words):

    angle_vector = np.array([])
    angle_min = 90

    dom1, A = rank_one_approx(possible_words)

    for i in range(0,len(possible_words)):
        angle1 = np.arccos(np.matmul(A[:,i],dom1)/(np.linalg.norm(A[:,i])))
        
        if np.isnan(angle1):
            angle1 = np.pi / 2
        
        angle_vector = np.append(angle_vector,angle1*180/np.pi)

        if np.abs(angle1) < angle_min:
            angle_min = angle1

    best_words = np.array(possible_words)[angle_vector.argsort()]
    angle_vector = angle_vector[angle_vector.argsort()]
    return best_words,angle_vector

def edit_possible_words(possible_words, green, yellow, grey):
    copy = possible_words
    for (x,y) in green:
        for word in possible_words[:]:
            if word[y] == x: 
                pass
            else:
                copy.remove(word)
    
    for (x,y) in yellow:
        for word in possible_words[:]:
            if x in word: 
                if word[y] == x: 
                    copy.remove(word)
                else:
                    pass
            else:
                copy.remove(word)
    
    for x in grey:
        for word in possible_words[:]:
            if x in word:
                copy.remove(word)
        
    return copy

def det_conds(guess, secret_word):
    #get greys
    grey = []
    for x in guess:
        if x in secret_word:
            pass
        else:
            grey.append(x)

    #get greens 
    green = []
    yellow = []
    for i in [0,1,2,3,4]:
        if guess[i] == secret_word[i]:
            green.append((guess[i], i))
        else:
            if guess[i] in grey:
                pass
            else:
                yellow.append((guess[i], i))

    return green, yellow, grey


In [13]:
find_best_guess(possible_words)

(array(['THEIR', 'THERE', 'THESE', ..., 'VULGO', 'ZUPPA', 'KULFI'],
       dtype='<U5'),
 array([31.10521866, 32.76730707, 41.26033808, ..., 89.99999713,
        89.99999729, 89.99999742]))

In [5]:
def guess_word(secret_word):
    global possible_words
    possible_words = all_wordle_words.copy()
    guess = ''
    count = 1
    while guess != secret_word:
        best_words, angle_vector = find_best_guess(possible_words)
        print('----------------------------------------------------')
        print('ATTEMPT NUMBER ' + str(count))
        if len(best_words) < 5:
            print('Top Guesses are : ')
            for i in range(0,len(best_words)):
                print(best_words[i] + ' ' + str(angle_vector[i]))
        else: 
            print('Top 5 Guesses are : ')
            for i in range(0,5):
                print(best_words[i] + ' ' + str(angle_vector[i]))
        guess = best_words[0]
        print('GUESS IS : ' + guess)
        print('----------------------------------------------------')
        green, yellow, grey = det_conds(guess, secret_word)
        possible_words = edit_possible_words(possible_words, green, yellow, grey)
        count += 1
    count -= 1
    print('Guessed in: ' + str(count) + ' attempts')

In [6]:
def guess_words(possible_words, secret_words):
    lst = []
    for secret_word in secret_words:
        guess = ''
        count = 1
        possible_words = all_wordle_words.copy()
        while guess != secret_word:
            best_words, angle_vector = find_best_guess(possible_words)
            print('----------------------------------------------------')
            print('ATTEMPT NUMBER ' + str(count))
            if len(best_words) < 5:
                print('Top Guesses are : ')
                for i in range(0,len(best_words)):
                    print(best_words[i] + ' ' + str(angle_vector[i]))
            else: 
                print('Top 5 Guesses are : ')
                for i in range(0,5):
                    print(best_words[i] + ' ' + str(angle_vector[i]))
            guess = best_words[0]
            print('GUESS IS : ' + guess)
            print('----------------------------------------------------')
            green, yellow, grey = det_conds(guess, secret_word)
            possible_words = edit_possible_words(possible_words, green, yellow, grey)
            count += 1
        count -= 1
        print('Guessed in: ' + str(count) + ' attempts')
        lst.append(count)
    return lst

In [7]:
guess_word('POUCH')

/var/folders/wb/k2nttrg10dv0x7r4j0tp7g0c0000gn/T/ipykernel_34282/1086611585.py:30: RuntimeWarning: invalid value encountered in double_scalars
  angle1 = np.arccos(np.matmul(A[:,i],dom1)/(np.linalg.norm(A[:,i])))


----------------------------------------------------
ATTEMPT NUMBER 1
Top 5 Guesses are : 
THERE 29.68709205186252
THESE 36.10945210389861
THEIR 36.74972690374557
THETE 38.779771227692734
THEME 39.04666173232942
GUESS IS : THERE
----------------------------------------------------
----------------------------------------------------
ATTEMPT NUMBER 2
Top 5 Guesses are : 
HAPPY 36.192364205585555
HUMPY 41.94117447438371
HUMAN 44.33516419918546
HAPLY 46.8621330528271
HAPPI 46.96339591383444
GUESS IS : HAPPY
----------------------------------------------------
----------------------------------------------------
ATTEMPT NUMBER 3
Top 5 Guesses are : 
PUNCH 2.255273326106609
PINCH 34.697711658000784
PILCH 51.567705456265735
POUCH 52.06815380592639
POOCH 52.250177673765556
GUESS IS : PUNCH
----------------------------------------------------
----------------------------------------------------
ATTEMPT NUMBER 4
Top Guesses are : 
POUCH 0.0
GUESS IS : POUCH
-------------------------------------